TODO: 
- M30/NGC whatever from Sollima (galstreams) and Ibata. 
- The Ibata stream orbit fits that failed should still appear in the table
- Stream widths

# Imports

In [2]:
import io
import pathlib
import pickle
import re

import ads
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
import galstreams
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from astropy.stats import median_absolute_deviation as MAD
from gala.units import galactic
from helpers import (
    get_full_galstreams_poly,
    get_isochrone,
    make_ibata_poly_nodes,
    run_orbit_fit,
    get_default_track_for_stream,
)
from pyia import GaiaData
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.optimize import minimize
from tqdm.auto import tqdm

%matplotlib inline

Fiducial isochrone model:

In [3]:
iso = get_isochrone()

tmp_iso = iso[iso["stage"] == 1]
MSTO_absmag = tmp_iso["rP1"].min()

In [4]:
mwstreams = galstreams.MWStreams()

Initializing galstreams library from master_log... 


# Load data

## Ibata:

In [5]:
# tbl = at.Table.read("../data/Ibata2023_Table1.csv")
ibata_g = GaiaData(
    "../data/Ibata2023_GaiaDR3-xm.fits",
    radial_velocity_colname="vh",
    radial_velocity_unit=u.km / u.s,
)

In [6]:
ibata_tbl = at.Table.read("../data/ibata_streams_full.csv")
sID_to_name = {row["sID"]: row["name"] for row in at.unique(ibata_tbl, keys="sID")}
name_to_sID = {v: k for k, v in sID_to_name.items()}

In [7]:
_dist = ibata_g.get_distance(allow_negative=True)
_dist[~np.isfinite(_dist)] = 100 * u.Mpc  # put them very far away if missing

ibata_c = ibata_g.get_skycoord(distance=_dist)

## S5 streams:
s5_streams = "300S",\n"Willka Yaku",\n"AAU", "Jet", "Phoenix", "Indus", "Palca", "Elqui", "Turranburra",

Distance modulus functions are from Li et al 2022, Table 1

In [8]:
rows = []

stream = mwstreams["Orphan-K19"]
track = stream.track.transform_to(stream.stream_frame.replicate_without_data())
rows.append(
    {
        "name": "OC",
        "gs_track": "Orphan-K19",
        "other_names": ["Orphan", "Chenab"],
        "phi1": track.phi1.wrap_at(180 * u.deg),
        "phi2": track.phi2,
        "sky_track": track,
        "sky_ref": "2019MNRAS.485.4726K",
        "distmod": track.distance.distmod.value,
        "dist_ref": "2019MNRAS.485.4726K",
        "mass": 4e6 * u.Msun,
        "mass_ref": "2019MNRAS.485.4726K",
    }
)

stream = mwstreams["300S-F18"]
track = stream.track.transform_to(stream.stream_frame)
track_distmod = 48.9952 - 0.2083 * stream.track.ra.degree
rows.append(
    {
        "name": "300S",
        "gs_track": "300S-F18",
        "other_names": [],
        "phi1": track.phi1.wrap_at(180 * u.deg),
        "phi2": track.phi2,
        "sky_track": coord.SkyCoord(
            phi1=track.phi1,
            phi2=track.phi2,
            distance=coord.Distance(distmod=track_distmod),
            frame=stream.stream_frame.replicate_without_data(),
        ),
        "sky_ref": "2018ApJ...866...42F",
        "distmod": track_distmod,
        "dist_ref": "2022ApJ...928...30L",
        "mass": 10**4.7 * u.Msun,
        "mass_ref": "2024MNRAS.tmp..241U",
    }
)

# ---
stream = mwstreams["Willka_Yaku-S18"]
track = stream.track.transform_to(stream.stream_frame)
track_distmod = np.full(len(track), 17.8)
rows.append(
    {
        "name": "Willka Yaku",
        "gs_track": "Willka_Yaku-S18",
        "other_names": ["Willka_Yaku"],
        "phi1": track.phi1.wrap_at(180 * u.deg),
        "phi2": track.phi2,
        "sky_track": coord.SkyCoord(
            phi1=track.phi1,
            phi2=track.phi2,
            distance=coord.Distance(distmod=track_distmod),
            frame=stream.stream_frame.replicate_without_data(),
        ),
        "sky_ref": "2018ApJ...862..114S",
        "distmod": track_distmod,
        "dist_ref": "2022ApJ...928...30L",
        "mass": 14e4 * u.Msun,
        "mass_ref": "2018ApJ...862..114S",  # shipp 2018
    }
)

# ---
stream = mwstreams["AAU-ATLAS-L21"]
track_phi1 = np.concatenate(
    (
        mwstreams["AAU-ATLAS-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi1.wrap_at(180 * u.deg)
        .degree,
        mwstreams["AAU-AliqaUma-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi1.wrap_at(180 * u.deg)
        .degree,
    )
)
track_phi2 = np.concatenate(
    (
        mwstreams["AAU-ATLAS-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi2.degree,
        mwstreams["AAU-AliqaUma-L21"]
        .track.transform_to(mwstreams["AAU-ATLAS-L21"].stream_frame)
        .phi2.degree,
    )
)
track_distmod = 16.67 - 0.034 * track_phi1
rows.append(
    {
        "name": "AAU",
        "gs_track": "AAU-ATLAS-L21",
        "other_names": ["ATLAS", "Aliqa Uma"],
        "phi1": track_phi1 * u.deg,
        "phi2": track_phi2 * u.deg,
        "sky_track": coord.SkyCoord(
            phi1=track_phi1 * u.deg,
            phi2=track_phi2 * u.deg,
            distance=coord.Distance(distmod=track_distmod),
            frame=mwstreams["AAU-ATLAS-L21"].stream_frame.replicate_without_data(),
        ),
        "sky_ref": "2021ApJ...911..149L",
        "distmod": track_distmod,
        "dist_ref": "2022ApJ...928...30L",
        "mass": (12e4 + 18e4) * u.Msun,  # Shipp+2018
        "mass_ref": "2018ApJ...862..114S",
    }
)

distmod_trends = {
    "Jet-F22": lambda x: 17.45 - 0.014 * x,
    "Phoenix-S19": lambda x: 16.26 + 0.008 * x,
    "Indus-S19": lambda x: 15.90 - 0.016 * x,
    "Palca-S18": lambda x: np.full_like(x, 17.8),
    "Elqui-S19": lambda x: 18.48 - 0.043 * x,
    "Turranburra-S19": lambda x: np.full_like(x, 17.1),
}
ref_to_bibcode = {
    "shipp2018": "2018ApJ...862..114S",
    "shipp2019": "2019ApJ...885....3S",
    "ferguson2022": "2022AJ....163...18F",
}
masses = {
    "Jet-F22": (2.5e4, "2018MNRAS.480.5342J"),
    "Phoenix-S19": (3e4, "2018ApJ...862..114S"),
    "Indus-S19": (650e4, "2018ApJ...862..114S"),
    "Palca-S18": (1e6, "2022A&A...660A..29T"),
    "Elqui-S19": (320e4, "2018ApJ...862..114S"),
    "Turranburra-S19": (180e4, "2018ApJ...862..114S"),
}
for key, trend in distmod_trends.items():
    stream = mwstreams[key]
    name = key.split("-")[0]
    track = stream.track.transform_to(stream.stream_frame.replicate_without_data())
    track_phi1 = track.phi1.wrap_at(180 * u.deg).degree
    rows.append(
        {
            "name": name,
            "gs_track": key,
            "other_names": [] if name != "Palca" else ["Cetus"],
            "phi1": track_phi1 * u.degree,
            "phi2": track.phi2,
            "sky_track": coord.SkyCoord(
                phi1=track.phi1,
                phi2=track.phi2,
                distance=coord.Distance(distmod=trend(track_phi1)),
                frame=stream.stream_frame,
            ),
            "sky_ref": ref_to_bibcode.get(stream.ref, stream.ref),
            "distmod": trend(track_phi1),
            "dist_ref": "2022ApJ...928...30L",
            "mass": masses[key][0] * u.Msun,
            "mass_ref": masses[key][1],
        }
    )

s5_data = at.QTable(rows)

/usr/local/lib64/python3.9/site-packages/astropy/units/quantity.py:502: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  value = np.array(value, dtype=dtype, copy=copy, order=order,


# Orbit fitting

We fit orbits to the Ibata streams to determine distance tracks:

In [9]:
data_file = pathlib.Path("../data/stream-orbit-fits.pkl")

if not data_file.exists():
    orbitfit_data = {}

    for sid in tqdm(np.unique(ibata_tbl["sID"])):
        print(sid, sID_to_name[sid])
        mask = ibata_tbl["sID"] == sid
        orbitfit_data[sid] = run_orbit_fit(sid, ibata_g[mask], ibata_c[mask])

    with open(data_file, "wb") as f:
        pickle.dump(orbitfit_data, f)

else:
    with open(data_file, "rb") as f:
        orbitfit_data = pickle.load(f)

In [10]:
# plot_path = pathlib.Path("../plots").resolve()
# plot_path.mkdir(exist_ok=True)

# for sid, this_data in all_data.items():
#     fig, _ = make_components_plot(
#         this_data["orbit"],
#         this_data["orbit_fr"],
#         this_data["c_fr"],
#         this_data["obj_data"],
#     )
#     fig.suptitle(sID_to_name[sid], fontsize=20)
#     fig.savefig(plot_path / f"stream-{sid:04d}-orbitfit.png", dpi=200)
#     plt.close(fig)

In [11]:
ibata_fit_tbl = at.Table(
    [
        {"sID": sid, "name": sID_to_name[sid], **d["p"]}
        for sid, d in orbitfit_data.items()
    ]
)

failed_sIDs = [
    21,
    28,
    48,  # Orphan
    77,
    80,
]
for num in failed_sIDs:
    print(num, sID_to_name[num])

ibata_fit_tbl = ibata_fit_tbl[~np.isin(ibata_fit_tbl["sID"], failed_sIDs)]

21 New-6
28 New-9
48 Orphan
77 NGC7099
80 New-25


# Data tables


## Internal data table: 
- Name
- sID in Ibata
- preferred galstreams track name
- sky track 
- dist/distmod track either from orbit fit or from source
- sky polygon
- stellar mass
- sources for all

Precedence is:
- S5
- Ibata atlas
- Galstreams


In [12]:
internal_data_rows = []

for tmp_row in s5_data:
    row = {}
    row["name"] = tmp_row["name"]
    row["other_names"] = tmp_row["other_names"]
    row["galstreams_track_name"] = tmp_row["gs_track"]

    row["track"] = tmp_row["sky_track"]
    row["sky_track_ref"] = tmp_row["sky_ref"]
    row["dist_track_ref"] = tmp_row["dist_ref"]

    row["stellar_mass"] = tmp_row["mass"]
    row["stellar_mass_ref"] = tmp_row["mass_ref"]

    # Sky polygon:
    if row["name"] == "AAU":
        # Special case: get sky track for AAU from Ibata member stars
        d = orbitfit_data[name_to_sID["ATLAS"]]
        _, row["sky_poly"] = make_ibata_poly_nodes(d["c_fr"])

    else:
        row["sky_poly"] = get_full_galstreams_poly(
            mwstreams[tmp_row["gs_track"]].poly_sc
        )

    internal_data_rows.append(row)

internal_data = at.QTable(internal_data_rows)

Now match S5 streams to Ibata atlas stream IDs:

In [13]:
ibata_names = np.asarray(np.unique(ibata_tbl["name"]))
ibata_sIDs = []
for row in internal_data:
    row_names = [row["name"]] + row["other_names"]
    for name in row_names:
        if name in ibata_names:
            sID = name_to_sID[name]
            break
    else:
        sID = 0
    ibata_sIDs.append(sID)
internal_data["ibata2023_sID"] = ibata_sIDs

In [14]:
ibata_to_galstreams = {
    "Tuc3": "TucanaIII",
    "NGC7099": "M30",
    "M5": "NGC5904",
    "M92": "NGC6341",
}

In [15]:
for name in ibata_names:
    sID = name_to_sID[name]
    if sID in internal_data["ibata2023_sID"]:
        continue

    row = {}
    row["name"] = name
    row["other_names"] = (
        [] if name not in ibata_to_galstreams else [ibata_to_galstreams[name]]
    )

    track_name, _ = get_default_track_for_stream(name)
    if track_name is None:
        track_name, _ = get_default_track_for_stream(
            ibata_to_galstreams.get(name, name)
        )
    row["galstreams_track_name"] = track_name if track_name is not None else ""
    row["ibata2023_sID"] = sID

    if sID in failed_sIDs or sID not in orbitfit_data:
        continue

    this_data = orbitfit_data[sID]
    sky_track, row["sky_poly"] = make_ibata_poly_nodes(this_data["c_fr"])

    x = this_data["orbit_fr"].phi1.degree
    spl = InterpolatedUnivariateSpline(
        x[np.argsort(x)], this_data["orbit_fr"].distance.kpc[np.argsort(x)], k=3
    )

    row["track"] = coord.SkyCoord(
        phi1=sky_track.phi1,
        phi2=sky_track.phi2,
        distance=spl(sky_track.phi1.degree) * u.kpc,
        frame=sky_track.frame.replicate_without_data(),
    )
    row["sky_track_ref"] = "2023arXiv231117202I"
    row["dist_track_ref"] = ""  # TODO: this paper

    # Estimate stellar mass using an isochrone:
    DM = np.mean(row["track"].distance.distmod.value)
    MSTO_r = DM + MSTO_absmag

    idx = np.where((iso["G"] + DM) < 20.0)[0]
    dIMF = iso["int_IMF"][idx[-1]] - iso["int_IMF"][idx[0]]
    M = 2.0 * len(d["c_fr"]) / dIMF

    row["stellar_mass"] = M * u.Msun
    row["stellar_mass_ref"] = ""  # TODO: this paper

    internal_data.add_row(row)

Initializing galstreams library from master_log... 


Now find all other streams in Galstreams that are not already in our internal table:

In [16]:
galstreams_masses = {
    "20.0-1": (1e5 * u.Msun, "mateu2018"),  # Estimate based on MV=-7.3,
    "Corvus": (1e5 * u.Msun, "mateu2018"),  # Estimate based on MV=-7.5,
    "PS1-A": (0.22e3 * 2 * u.Msun, "bernard2016"),  # M/L = 2
    "PS1-B": (1.2e3 * 2 * u.Msun, "bernard2016"),
    "PS1-C": (1e3 * 2 * u.Msun, "bernard2016"),
    "PS1-D": (7.5e3 * 2 * u.Msun, "bernard2016"),
    "PS1-E": (0.5e3 * 2 * u.Msun, "bernard2016"),
    "Pal13": (1.4e3 * 2 * u.Msun, "shipp2020"),  # M/L = 2, Lv in tails from RRL
    "Pegasus": (2e3 * 2 * u.Msun, "perottoni2019"),  # M/L = 2
    "Ravi": (1e4 * u.Msun, "shipp2018"),
    "Turbio": (3.5e3 * u.Msun, "shipp2018"),
    "Wambelong": (1.6e3 * u.Msun, "shipp2018"),
}

# No reported masses:
# Grillmair 2009: Acheron, Cocytos, Lethe, Styx
# Grillmair 2013: Alpheus
# Williams+2011: Aquarius
# Ibata+2021: C-4, C-5, C-8, Gaia-2, Gaia-3, Gaia-4, Gaia-5, Gunnthra, M2
# Myeong+2017: Eridanus, Pal15
# Grillmair 2014: Hermus, Hyllus
# Sollima 2020: M30, NGC6362
# Grillmair 2017a: Sangarius, Scamander
# Grillmair 2017b: Murrumbidgee, Molonglo, Orinoco, Kwando
# Bonaca+2012: Tri-Pis
# Weiss+2018: Parallel, Perpendicular

In [17]:
gs_to_internal_name = {
    "AAU-ATLAS": "AAU",
    "AAU-AliqaUma": "AAU",
    "M68-Fjorm": "Fjorm",
    "NGC3201-Gjoll": "Gjoll",
    "OmegaCen-Fimbulthul": "Fimbulthul",
    "Orphan-Chenab": "OC",
    "TucanaIII": "Tuc3",
    "Jhelum-a": "Jhelum",
    "Jhelum-b": "Jhelum",
}

galstreams_default_tracks = np.array(list(mwstreams.keys()))
galstreams_names = np.array(
    [
        x
        for x in mwstreams.all_unique_stream_names()
        if x not in ["Cetus-New", "Cetus-Palca", "ACS", "Monoceros", "Sagittarius"]
    ]
)

_all_internal_names = np.concatenate(
    (internal_data["name"], [x for y in internal_data["other_names"] for x in y])
)
for name in galstreams_names:
    if gs_to_internal_name.get(name, name) in _all_internal_names:
        continue

    row = {}
    row["name"] = name
    row["other_names"] = []

    track_name, stream = get_default_track_for_stream(name)
    row["galstreams_track_name"] = track_name if track_name is not None else ""

    row["sky_poly"] = get_full_galstreams_poly(stream.poly_sc)
    row["track"] = stream.track.transform_to(
        stream.stream_frame.replicate_without_data()
    )
    row["sky_track_ref"] = stream.ref
    row["dist_track_ref"] = stream.ref

    if name in galstreams_masses:
        row["stellar_mass"] = galstreams_masses[name][0]
        row["stellar_mass_ref"] = galstreams_masses[name][1]
    else:
        row["stellar_mass"] = np.nan * u.Msun
        row["stellar_mass_ref"] = ""

    internal_data.add_row(row)

Final adjustments to fill in values in columns:

In [18]:
paper_to_bibcode = {
    "bernard2016": "2016MNRAS.463.1759B",
    "bonaca2012": "2012ApJ...760L...6B",
    "grillmair2009": "2009ApJ...693.1118G",
    "grillmair2013": "2013ApJ...769L..23G",
    "grillmair2014": "2014ApJ...790L..10G",
    "grillmair2017": "2017ApJ...834...98G",
    "grillmair2017b": "2017ApJ...847..119G",
    "ibata2021": "2021ApJ...914..123I",
    "malhan2018": "2018MNRAS.481.3442M",
    "mateu2018": "2018MNRAS.474.4112M",
    "myeong2017": "2017MNRAS.469L..78M",
    "perottoni2019": "2019MNRAS.486..843P",
    "shipp2018": "2018ApJ...862..114S",
    "shipp2020": "2020AJ....160..244S",
    "sollima2020": "2020MNRAS.495.2222S",
    "weiss2018": "2018ApJ...867L...1W",
    "williams2011": "2011ApJ...728..102W",
}
for colname in ["sky_track_ref", "dist_track_ref", "stellar_mass_ref"]:
    internal_data[colname] = [
        paper_to_bibcode.get(x, x) for x in internal_data[colname]
    ]

## Discovery methods

In [19]:
discovery_method = {
    "Acheron": "photometry",
    "Styx": "photometry",
    "Cocytos": "photometry",
    "Lethe": "photometry",
    "Aquarius": "photometry",
    "Tri-Pis": "photometry",
    "Alpheus": "photometry",
    "Hermus": "photometry",
    "Hyllus": "photometry",
    "PS1-C": "photometry",
    "PS1-D": "photometry",
    "PS1-E": "photometry",
    "PS1-A": "photometry",
    "PS1-B": "photometry",
    "Orinoco": "photometry",
    "Sangarius": "photometry",
    "Scamander": "photometry",
    "Molonglo": "photometry",
    "Murrumbidgee": "photometry",
    "Pal15": "photometry",
    "Eridanus": "photometry",
    "Wambelong": "photometry",
    "Ravi": "photometry",
    "Palca": "photometry",
    "Willka Yaku": "photometry",
    "Turbio": "photometry",
    "300S": "photometry",
    "Parallel": "photometry",
    "Perpendicular": "photometry",
    "Corvus": "rrl",
    "20.0-1": "rrl",
    "Gaia-5": "streamfinder",
    "Gaia-4": "streamfinder",
    "Gaia-3": "streamfinder",
    "Turranburra": "photometry",
    "Elqui": "photometry",
    "Indus": "photometry",
    "Phoenix": "photometry",
    "OC": "photometry",
    "Pegasus": "photometry",
    "Pal13": "photometry",
    "M30": "photometry",
    "NGC6362": "astrometry",
    "AAU": "photometry",
    "M2": "streamfinder",
    "Gaia-2": "streamfinder",
    "C-8": "streamfinder",
    "C-5": "streamfinder",
    "C-4": "streamfinder",
    "Gunnthra": "streamfinder",
    "Jet": "photometry",
    "Gjoll": "streamfinder",
    "Gaia-10": "streamfinder",
    "Hrid": "streamfinder",
    "Hydrus": "photometry",
    "Gaia-8": "streamfinder",
    "Gaia-7": "streamfinder",
    "Gaia-6": "streamfinder",
    "Ylgr": "streamfinder",
    "Tuc3": "photometry",
    "Gaia-12": "streamfinder",
    "Gaia-11": "streamfinder",
    "Gaia-9": "streamfinder",
    "Kwando": "photometry",
    "GD-1": "photometry",
    "C-10": "streamfinder",
    "C-11": "streamfinder",
    "C-12": "streamfinder",
    "C-13": "streamfinder",
    "C-19": "streamfinder",
    "C-20": "streamfinder",
    "C-22": "streamfinder",
    "Gaia-1": "streamfinder",
    "C-23": "streamfinder",
    "C-25": "streamfinder",
    "C-7": "streamfinder",
    "C-9": "streamfinder",
    "Fimbulthul": "streamfinder",
    "Fimbulthul-S": "streamfinder",
    "Sylgr": "streamfinder",
    "Fjorm": "streamfinder",
    "C-24": "streamfinder",
    "Svol": "streamfinder",
    "Slidr": "streamfinder",
    "Phlegethon": "streamfinder",
    "New-11": "streamfinder",
    "New-10": "streamfinder",
    "New-1": "streamfinder",
    "NGC7492": "streamfinder",
    "NGC7089": "streamfinder",
    "NGC6397": "streamfinder",
    "NGC6101": "streamfinder",
    "NGC5466": "photometry",
    "NGC288": "astrometry",
    "NGC2808": "streamfinder",
    "NGC2298": "astrometry",
    "NGC1851": "streamfinder",
    "NGC1261b": "streamfinder",
    "NGC1261a": "streamfinder",
    "NGC1261": "streamfinder",
    "M92": "astrometry",
    "M5": "photometry",
    "Leiptr": "streamfinder",
    "LMS-1": "iom-clustering",
    "New-12": "streamfinder",
    "Kshir": "streamfinder",
    "New-13": "streamfinder",
    "New-15": "streamfinder",
    "Pal5": "photometry",
    "Ophiuchus": "photometry",
    "New-8": "streamfinder",
    "New-7": "streamfinder",
    "New-5": "streamfinder",
    "New-4": "streamfinder",
    "New-3": "streamfinder",
    "New-28": "streamfinder",
    "New-27": "streamfinder",
    "New-26": "streamfinder",
    "New-24": "streamfinder",
    "New-23": "streamfinder",
    "New-22": "streamfinder",
    "New-21": "streamfinder",
    "New-20": "streamfinder",
    "New-2": "streamfinder",
    "New-19": "streamfinder",
    "Jhelum": "photometry",
    "New-16": "streamfinder",
    "New-14": "streamfinder",
    "New-17": "streamfinder",
}
discovery_method = at.Table(
    {
        "name": list(discovery_method.keys()),
        "discovery_method": list(discovery_method.values()),
    }
)

In [20]:
internal_data = at.join(internal_data, discovery_method, keys="name", join_type="left")

In [21]:
with open("../data/all_stream_summary_data.pkl", "wb") as f:
    pickle.dump(internal_data, f)

## Main Summary Table

- name
- phi1,2 = (0, 0) in ICRS coords
- length
- distmod (dist)
- range of distmod (dist)
- stellar mass
- found with gaia y/n
- spec follow up y/n
- associated with progenitor y/n

In [26]:
summary_table = []
for irow in internal_data:
    row = {}
    row["Name"] = irow["name"]

    frame = irow["track"].frame.replicate_without_data()

    _sort_idx = np.argsort(irow["track"].phi1.degree)
    dist_spl = InterpolatedUnivariateSpline(
        irow["track"].phi1.degree[_sort_idx], irow["track"].distance.kpc[_sort_idx], k=3
    )
    origin_c = coord.SkyCoord(
        phi1=0 * u.deg, phi2=0 * u.deg, distance=np.abs(dist_spl(0.0)) * u.kpc, frame=frame
    )
    origin_c = origin_c.transform_to(coord.ICRS())
    galcen = origin_c.transform_to(coord.Galactocentric())
    row["Origin RA"] = origin_c.ra
    row["Origin Dec."] = origin_c.dec

    # dhelio_name = "Origin Helio. Dist."
    dhelio_name = r"$D_{\rm hel}$"
    row[dhelio_name] = origin_c.distance.view(u.Quantity)

    # rgal_name = "Origin Gal. Radius"
    rgal_name = r"$r_{\rm gal}$"
    row[rgal_name] = np.squeeze(np.linalg.norm(galcen.data.xyz))

    # if np.isclose(irow["track"].distance.kpc.min(), irow["track"].distance.kpc.max()):
    #     row["Min. Dist."] = np.nan * u.kpc
    #     row["Max. Dist."] = np.nan * u.kpc
    # else:
    #     row["Min. Dist."] = irow["track"].distance.min().view(u.Quantity)
    #     row["Max. Dist."] = irow["track"].distance.max().view(u.Quantity)

    row["Length"] = irow["track"].phi1.max() - irow["track"].phi1.min()
    row["Width"] = 0.0 * u.deg  # TODO
    row["Reference"] = irow["sky_track_ref"]

    row["Stellar Mass"] = irow["stellar_mass"]
    if irow["stellar_mass_ref"] != "" or np.isnan(irow["stellar_mass"]):
        row["Stellar Mass Ref."] = irow["stellar_mass_ref"]
    else:
        row["Stellar Mass Ref."] = "this work"

    if irow["ibata2023_sID"] != 0:
        row["Distance Ref."] = "this work"
    else:
        row["Distance Ref."] = irow["sky_track_ref"]

    try:
        pm1 = irow["track"].pm_phi1_cosphi2
        pm_track_bool = np.any(~np.isclose(pm1.value, 0))
    except TypeError:
        pm_track_bool = False

    # Special case some:
    if irow["name"] in ["Aquarius"]:
        pm_track_bool = False
    elif irow["name"].startswith("Gaia"):
        pm_track_bool = True

    row["Gaia"] = irow["ibata2023_sID"] != 0 or pm_track_bool
    row["S5"] = np.in1d([irow["name"]] + irow["other_names"], s5_data["name"]).any()
    summary_table.append(row)

tmp_summary_table = at.QTable(summary_table)

Sort by: S5, Ibata, all remaining, and then alphabetical by name within each:

In [27]:
# Custom sort function:
def natural_sort(key):
    return [int(s) if s.isdigit() else s for s in re.split(r"(\d+)", key)]


tbl1 = tmp_summary_table[tmp_summary_table["S5"]]
tbl2 = tmp_summary_table[tmp_summary_table["Gaia"] & ~tmp_summary_table["S5"]]
tbl3 = tmp_summary_table[~tmp_summary_table["Gaia"] & ~tmp_summary_table["S5"]]
assert len(tbl1) + len(tbl2) + len(tbl3) == len(tmp_summary_table)
tbls = [tbl1, tbl2, tbl3]
table_heads = [
    "Observed by S5 Survey",
    "Detected in Gaia",
    "Needs Follow-up Observations",
]

sorted_tbls = []
split_streams = []
for tbl in tbls:
    sorted_names = sorted(tbl["Name"], key=natural_sort)
    argsort = np.array([np.where(tbl["Name"] == name)[0][0] for name in sorted_names])
    tbl = tbl[argsort]
    sorted_tbls.append(tbl)
    split_streams.append(tbl["Name"][-1])

summary_table = at.vstack(sorted_tbls)
summary_table.remove_columns(["S5", "Gaia"])

In [28]:
bibcode_to_bib = {}
bibcode_to_ref_id = {}
i = 1
for bibcode in np.unique(
    np.concatenate((summary_table["Reference"], summary_table["Stellar Mass Ref."]))
):
    if bibcode == "":
        continue

    try:
        paper = list(ads.SearchQuery(bibcode=bibcode))[0]
    except IndexError:
        print(bibcode)
        continue

    # Handle grillmair 2017b
    cite_key = f"{paper.first_author.split(',')[0].lower()}:{paper.year}"
    if cite_key in [x[0] for x in bibcode_to_bib.values()]:
        cite_key = f"{cite_key}b"
    bibtex = paper.bibtex.replace(bibcode, cite_key)
    bibcode_to_bib[bibcode] = (cite_key, bibtex)
    bibcode_to_ref_id[bibcode] = i
    i += 1

bibcode_to_ref_id["this work"] = "*"

/usr/local/lib/python3.9/site-packages/ads/base.py:135: RuntimeWarning: No token found
  warnings.warn("No token found", RuntimeWarning)


APIResponseError: '{\n  "error": "Unauthorized"\n}\n'

Replace references with letters after the stream name:

In [24]:
# Add reference letters next to values:
# with_refs = {"Name": [], dhelio_name: [], "Stellar Mass": []}
# for row in summary_table:
#     ref = bibcode_to_ref_id.get(row["Reference"], "")
#     with_refs["Name"].append(f"{row['Name']}$^{{({ref})}}$" if ref else "")

#     d = row[dhelio_name]
#     ref = bibcode_to_ref_id.get(row["Distance Ref."], "")
#     with_refs[dhelio_name].append(
#         f"{d.value:.1f}$^{{({ref})}}$" if ref else f"{d.value:.1f}"
#     )

#     ref = bibcode_to_ref_id.get(row["Stellar Mass Ref."], "")
#     with_refs["Stellar Mass"].append(
#         f"{row['Stellar Mass'].value:.0e}$^{{({ref})}}$"
#         if ref
#         else f"{row['Stellar Mass'].value:.0e}"
#     )

# summary_table["Name"] = with_refs["Name"]
# summary_table[dhelio_name] = with_refs[dhelio_name]
# summary_table[dhelio_name].unit = u.kpc
# summary_table["Stellar Mass"] = with_refs["Stellar Mass"]
# summary_table["Stellar Mass"].unit = u.Msun
# summary_table.remove_columns(["Reference", "Stellar Mass Ref.", "Distance Ref."])

# Add a new column with all reference letters:
refs = []
for row in summary_table:
    ref = bibcode_to_ref_id.get(row["Reference"], "")
    ref_d = bibcode_to_ref_id.get(row["Distance Ref."], "")
    ref_m = bibcode_to_ref_id.get(row["Stellar Mass Ref."], "")
    refs.append(",".join([f"({r})" for r in np.unique([ref, ref_d, ref_m]) if r]))

final_summary_table = summary_table.copy()
final_summary_table["References"] = refs
final_summary_table.remove_columns(["Reference", "Stellar Mass Ref.", "Distance Ref."])

In [25]:
caption = [f"({num}): \citet{{{bibcode_to_bib[bibcode][0]}}}" if bibcode in bibcode_to_bib else f"({num}): this work" for bibcode, num in bibcode_to_ref_id.items()]
caption = ", ".join(caption)

Split the full table into batches:

In [26]:
idx = np.arange(0, len(final_summary_table) + 1, 36)
if idx[-1] < len(final_summary_table):
    idx = np.concatenate((idx, [len(final_summary_table)]))

sub_tables = [final_summary_table[i:j] for i, j in zip(idx[:-1], idx[1:])]
[len(x) for x in sub_tables]

[36, 36, 36, 21]

In [27]:
texts = []
for i, tbl in enumerate(sub_tables):
    with io.StringIO() as f:
        tbl.write(
            f,
            format="ascii.latex",
            overwrite=True,
            formats={
                "Origin RA": "%.3f",
                "Origin Dec.": "%.3f",
                dhelio_name: "%.1f",
                rgal_name: "%.1f",
                "Length": "%.0f",
                "Width": "%.1f",
                "Stellar Mass": "%.0e",
            },
            latexdict={
                "header_start": r"\hline \hline",
                "data_start": r"\hline",
                "data_end": r"\hline \hline",
            },
            col_align="cSSSSSSccc"
        )

        text = f.getvalue()
        text = text.replace(" nan ", " ")

        text = re.sub("(\d+)e\+0(\d+)", r"$\1 \\times 10^{\2}$", text)
        # text = text.replace("_", "")
        texts.append(text)

In [28]:
unit_to_siunitx = {
    u.deg: "\\unit{\\degree}",
    u.kpc: "\\unit{\\kilo\\parsec}",
    u.Msun: "\\unit{\\Msun}",
}

In [29]:
for i, text in enumerate(texts, start=1):
    for bibcode, (cite_key, bibtex) in bibcode_to_bib.items():
        text = text.replace(bibcode, f"\\citet{{{cite_key}}}")

    split_text = text.split("\n")

    # Fix the column names with periods for aligning on decimal:
    # TODO: change replace to rstrip
    new_header = " & ".join(
        [f"{{{x.strip()}}}" for x in split_text[3].rstrip("\\").split(" & ")]
    )
    split_text[3] = new_header + "\\\\"

    # Fix the units to use siunitx:
    new_units = " & ".join(
        [
            unit_to_siunitx.get(col.unit, "")
            for col in final_summary_table.columns.values()
        ]
    )
    split_text[4] = new_units + "\\\\"

    # Table head for first block:
    if i == 1:
        split_text[5] = split_text[5] + "\\\\"
        head = table_heads[0]
        split_text.insert(
            6,
            f"\\multicolumn{{{len(final_summary_table.colnames)}}}{{l}}"
            f"{{\\bf {head}:}}\\\\[1pt]",
        )

    # Add hlines below any lines that start with the "split stream" names from above:
    lines = []
    for line in split_text:
        lines.append(line)
        for name, head in zip(split_streams[:-1], table_heads[1:]):
            if line.strip().startswith(name):
                lines.append("\\hline \\\\")
                lines.append(
                    f"\\multicolumn{{{len(final_summary_table.colnames)}}}{{l}}"
                    f"{{\\bf {head}:}}\\\\[1pt]"
                )

    text = "\n".join(lines)

    with open(f"../tex/figures/stream-summary-table-{i}.tex", "w") as f:
        f.write(text)

In [30]:
print(caption)

(1): \citet{grillmair:2009}, (2): \citet{williams:2011}, (3): \citet{bonaca:2012}, (4): \citet{grillmair:2013}, (5): \citet{grillmair:2014}, (6): \citet{bernard:2016}, (7): \citet{grillmair:2017}, (8): \citet{grillmair:2017b}, (9): \citet{myeong:2017}, (10): \citet{shipp:2018}, (11): \citet{fu:2018}, (12): \citet{weiss:2018}, (13): \citet{mateu:2018}, (14): \citet{jethwa:2018}, (15): \citet{malhan:2018}, (16): \citet{shipp:2019}, (17): \citet{koposov:2019}, (18): \citet{perottoni:2019}, (19): \citet{shipp:2020}, (20): \citet{sollima:2020}, (21): \citet{li:2021}, (22): \citet{ibata:2021}, (23): \citet{thomas:2022}, (24): \citet{ferguson:2022}, (25): \citet{ibata:2023}, (26): \citet{usman:2024}, (*): this work


In [31]:
with open("../tex/refs.bib") as f:
    refs_text = f.read()


for cite_key, bibtex in bibcode_to_bib.values():
    if cite_key in refs_text:
        continue
    else:
        print(cite_key)
        # print(bibtex)